Note: All code in script is sourced from various internet sources, from mentor Tomasz Popiel and from Alan Chalk at Sabre Insurance.

## Data manipulation

Contents:

 - Start_: import modules, set directories, load data ('01_df_all.pickle')
 
 - Omit the identical and all 0 columns
 
 - Deal with missing values 
 
 - One hot encoding categorical variables 
 
 - Deal with hccv's (a_v7 and a_v8)
 
 - Defining variables and save clean data ('02_df_all.pickle')

Notes:

 - remove 144 records where a_v6 is missing

### Start_:

Import modules

In [1]:
import os
import pickle
import pandas as pd

Set directories

In [2]:
print(os.getcwd())

dirRawData = "../RawData/"
dirPData = "../PData/"
dirPOutput = "../POutput/"

/Users/hujuiting/Desktop/QM-BA/GROUP PROJECT/PCode


Load data

In [3]:
fname = dirPData + '01_df_all.pickle'
with open(fname, 'rb') as f:
    dict_ = pickle.load(f)


# %% Load data

df_all = dict_['df_all']
del fname
df_all.shape

(65340, 347)

### Omit variables

Omit duplicate variables and variables with all values 0

 - tq_db_26 is duplicate of tq_db_25    
 - tq_db_44 is duplicate of tq_db_43    
 - tq_db33_13 and tq_db34_13 values are all 0

In [4]:
df_all.drop(['tq_db26_1','tq_db26_2','tq_db33_13','tq_db34_13','tq_db44_1','tq_db44_2','tq_db44_3'], axis=1, inplace=True)

### Deal with missing values

Check for missing values

In [5]:
srs_nulls = df_all.isnull().sum()
srs_nulls[srs_nulls > 0]

tq_v3_mean          1
tq_db11_std      3693
tq_db12_std      3693
tq_db13_std      3693
tq_db14_std      3693
tq_dt1_std       3693
tq_dt2_std       3693
tq_dt3_std       3693
tq_dt4_std       3693
tq_db15_std      3693
tq_db16_std      3693
tq_db17_std      3693
tq_db18_std      3693
tq_db19_std      3693
tq_db20_std      3693
tq_db21_std      3693
tq_db22_std      3693
tq_db23_std      3693
tq_v3_std        3693
tq_da1_Mean       370
tq_db4_Mean       366
tq_db5_Mean       366
tq_db6_Mean       366
tq_db7_Mean       366
tq_db8_Mean       365
tq_db9_Mean       365
tq_db10_Mean     3503
tq_v2_Mean         13
tq_da1_Mode      6980
tq_db1_Mode      4148
                ...  
tq_db46_17      65339
tq_db47_2        6584
tq_db47_3       12819
tq_db47_4       19345
tq_db47_5       26138
tq_db47_6       33333
tq_db47_7       41953
tq_db47_8       52425
tq_db47_9       61329
tq_db47_10      65334
tq_db48_2        6379
tq_db48_3       12246
tq_db48_4       18003
tq_db48_5       24196
tq_db48_6 

Deal with a_v6 missings 

-  delete data entries where a_v6 is missing because they were included in error and are not real objects

In [6]:
df_all.dropna(axis=0, subset=['a_v6'], inplace=True)
df_all.reset_index(drop=True, inplace=True)
print(df_all.shape)

(65196, 340)


Deal with histogram variables missing values 

- Replace missing values in histogram variables with 0

In [7]:
vars_hist_prefix = ('tq_db24', 'tq_db25', 'tq_db27', 'tq_db28', 'tq_db29', 'tq_db30','tq_db31',
                           'tq_v4', 'tq_v5', 'tq_db32', 'tq_db33', 'tq_db34', 
                           'tq_da12','tq_da13', 'tq_da14',
                           'tq_db35', 'tq_db36','tq_db37', 'tq_db38',
                           'tq_db39', 'tq_db40', 'tq_db41', 'tq_db42',
                           'tq_db43', 'tq_db45', 'tq_db46',
                           'tq_db47', 'tq_db48', 'tq_db49', 'tq_db50')

In [8]:
vars_hist = [col for col in df_all.columns if col.startswith(vars_hist_prefix)]

In [9]:
for column in vars_hist:
    df_all[column].fillna(0, inplace=True)

Deal with numerical variables missing values 

- Replace missing values in numerical variables with mean value

In [10]:
vars_numeric = [var for var in df_all.columns if var.endswith(('_mean', '_std', '_Mean', '_Mode', '_StdDev','_NMiss'))]

In [11]:
for column in vars_numeric:
    df_all[column].fillna((df_all[column].mean()), inplace=True)

In [12]:
for col in ['a_v9', 'a_v10', 'a_v11', 'a_v12']:
    
    df_all[col].fillna((df_all[col].mean()), inplace=True)

## One hot encoding categorical variables
One hot encoding for categorical variables: a_v1, a_v2, a_v3, a_v4, a_v5 

In [13]:
for var in ['a_v1', 'a_v2', 'a_v3', 'a_v4', 'a_v5']:
    df_onehot = pd.get_dummies(df_all[var], prefix = var + '_') 
    df_all = df_all.join(df_onehot)
    df_all.drop([var], axis=1, inplace=True)

## Deal with hccv's 

a_v7 and a_v8 are hccv's (a_v8 is a lower hierarchy under a_v7)

Concatenate a_v7 and a_v8, creating new variable a_v7_8 

In [14]:
df_all['a_v7_8'] = df_all['a_v7'].map(str) + '_' + df_all['a_v8'].map(str)

Count the number of each category in new variables a_v7_8

In [15]:
df2 = df_all['a_v7_8'].value_counts()

# Make the counts a data frame df2
df2 = pd.DataFrame(data=df2)

In [16]:
# To get the cumulative percentage of each category in new varaible a_v7_8
df2['Cumulative_Percentage'] = 100*df2.a_v7_8.cumsum()/df2.a_v7_8.sum()

listing out all the catogories with freq <= 24
 - one hot the whole feature
 - sum the columns in the resulting matrix and keep only cols with sum >= 25
 - add one final column "other"

One hot encode new variables a_v7_8

In [17]:
# one hot the whole varaible a_v7_8
df2 = pd.get_dummies(df_all['a_v7_8'], prefix = 'a_v7_8' + '_') 

# sum the varaibles in the resulting matrix
df2_freq = df2.sum(axis=0)

# keep only varaibles with frequency <= 24
idx_freq = df2_freq.values >= 25
df2 = df2.iloc[:, idx_freq]

# add varaible "other"
df2['a_v7_8_other'] = 1 - df2.sum(axis=1)

df_all = df_all.join(df2)

In [18]:
# Drop variables a_v7_8, av7 and av8
df_all.drop(['a_v7_8'], axis=1, inplace=True)
df_all.drop(['a_v7'], axis=1, inplace=True)
df_all.drop(['a_v8'], axis=1, inplace=True)

In [19]:
df_all.shape

(65196, 643)

In [20]:
df_all.head()

,tq_db11_mean,tq_db12_mean,tq_db13_mean,tq_db14_mean,tq_dt1_mean,tq_dt2_mean,tq_dt3_mean,tq_dt4_mean,tq_db15_mean,tq_db16_mean,...,a_v7_8__97_431,a_v7_8__97_46,a_v7_8__97_647,a_v7_8__97_770,a_v7_8__97_875,a_v7_8__97_909,a_v7_8__99_37,a_v7_8__99_995,a_v7_8__99_996,a_v7_8_other
0,14.6561,5.8498,1.4466,0.0079,0.1502,0.1146,0.0079,0.0040,0.0514,0.0593,...,0,0,0,0,0,0,0,0,0,0
1,16.4545,12.6364,1.7273,0.0000,0.0909,0.0909,0.0000,0.0000,0.0909,0.0909,...,0,0,0,0,0,0,0,0,0,0
2,15.6510,5.9688,1.4531,0.0052,0.1146,0.0938,0.0104,0.0104,0.1927,0.2083,...,0,0,0,0,0,0,0,0,0,0
3,15.3430,6.0233,1.3605,0.1628,0.0872,0.0814,0.0058,0.0233,0.0465,0.0581,...,0,0,0,0,0,0,0,0,0,0
4,13.7838,6.0270,1.5203,0.0000,0.0743,0.0676,0.0000,0.0000,0.0270,0.0405,...,0,0,0,0,0,0,0,0,0,0


## Defining variables and save clean data

In [21]:
# Defining category variables
vars_cat = ('a_v1_', 'a_v2_', 'a_v3_', 'a_v4_', 'a_v5_','a_v7_8_')
vars_cat = [col for col in df_all.columns if col.startswith(vars_cat)]

# Defining target variables 
vars_target = ['tq_dt1_mean','tq_dt2_mean','tq_dt3_mean','tq_dt4_mean','tq_dt1_std','tq_dt2_std','tq_dt3_std','tq_dt4_std']


### Save

In [22]:
# update df_all in dict_
dict_['df_all'] = df_all

# Target variables 
dict_['vars_target'] = vars_target

# Category variables
dict_['vars_cat'] = vars_cat

# Histogram variables 
dict_['vars_hist'] = vars_hist

# Numeric variables 
dict_['vars_numeric'] = vars_numeric + ['tq_v3','a_v6', 'a_v9', 'a_v10', 'a_v11', 'a_v12']

fname = dirPData + '02_df_all.pickle'
with open(fname, 'wb') as f:
    pickle.dump(dict_, f)

del dict_, fname